In [1]:
# from LogisticRegression import LogisticRegression
from CCS import CCS
from IPython.display import display
from datasets import load_dataset
from huggingface_hub import login
from jinja2 import Environment, PackageLoader, select_autoescape
from pprint import pp
from random import shuffle as rshuffle
from torch.utils.data import Dataset, random_split, DataLoader, TensorDataset
from tqdm import tqdm
from transformers import LlamaForCausalLM, LlamaTokenizer
import lightning as pl
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import torch
import torch.nn as nn
import os

In [2]:
VERBOSE = False
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pp(device)

login(add_to_git_credential=True)
np_rand = np.random.default_rng(seed=100500)
model_type = torch.bfloat16
pt_template = "ggplot2"
EXAMPLES_PER_QUESTION = 2
llama_size = '13b'

device(type='cuda')


In [3]:
# Load model
tokenizer = LlamaTokenizer.from_pretrained(
    f"meta-llama/Llama-2-{llama_size}-chat-hf",
    # device_map=device,
)
tokenizer.add_special_tokens({"pad_token": "<pad>"})
# tokenizer.pad_token = tokenizer.eos_token

hf_model = LlamaForCausalLM.from_pretrained(
    f"meta-llama/Llama-2-{llama_size}-chat-hf",
    torch_dtype=model_type,
    device_map=device,
    low_cpu_mem_usage=True,
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
hf_model.eval()
pp(hf_model)
pp(hf_model.config)
with torch.no_grad():
    pp(
        tokenizer.batch_decode(
            hf_model.generate(
                tokenizer("The capital of Russia is", return_tensors="pt").input_ids.to(
                    device
                ),
                max_length=20,
            )
        )[0]
    )

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 5120)
    (layers): ModuleList(
      (0-39): 40 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=5120, out_features=5120, bias=False)
          (k_proj): Linear(in_features=5120, out_features=5120, bias=False)
          (v_proj): Linear(in_features=5120, out_features=5120, bias=False)
          (o_proj): Linear(in_features=5120, out_features=5120, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=5120, out_features=13824, bias=False)
          (up_proj): Linear(in_features=5120, out_features=13824, bias=False)
          (down_proj): Linear(in_features=13824, out_features=5120, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head):

In [5]:
true_ids = [5574, 5852, 1565, 3009]
false_ids = [7700, 8824, 2089, 4541]
print(" ".join(f"'{tokenizer.decode(id_)}'" for id_ in true_ids))
print(" ".join(f"'{tokenizer.decode(id_)}'" for id_ in false_ids))

'True' 'True' 'true' 'true'
'False' 'False' 'false' 'false'


In [6]:
truthfulqa = load_dataset("truthful_qa", "generation")  # 817 rows
env = Environment(loader=PackageLoader("utils"), autoescape=select_autoescape())

In [7]:
# Datasets generators:


def div_items(items, max_num):
    if max_num is not None:
        for take_correct in (True, False):
            div = max_num // 2
            rshuffle(items)
            for e in items:
                if div == 0:
                    break
                if e["is_correct"] == take_correct:
                    div -= 1
                    yield e
    else:
        yield from items


def one_statement_ds_generator(max_per_q=None):
    """ Generator for a dataset with one statement per question. """

    with torch.no_grad():
        template = env.get_template("question_answer.jinja")
        counter = 0
        for row_id, row in tqdm(list(enumerate(truthfulqa["validation"]))):
            inc_as = row["incorrect_answers"]
            cor_as = row["correct_answers"]
            counter += 1
            items = [
                {
                    "id": counter,
                    "question_id": row_id,
                    "row": row,
                    "template_render_fn": template.render,
                    "is_correct": is_correct,
                    "template_render_params": {
                        "question": row["question"],
                        "answer": ans,
                    },
                }
                for is_correct in (True, False)
                for ans in (inc_as, cor_as)[is_correct]
            ]
            yield from div_items(items, max_per_q)


def conj_ds_generator(max_per_q=None):
    """ Generator for a dataset with two statements in conjunction per question. """

    with torch.no_grad():
        template = env.get_template("question_answers.jinja")
        counter = 0
        for row_id, row in tqdm(list(enumerate(truthfulqa["validation"]))):
            inc_as = row["incorrect_answers"]
            cor_as = row["correct_answers"]

            def create_yield(is_correct, answers):
                nonlocal counter
                counter += 1
                return {
                    "id": counter,
                    "question_id": row_id,
                    "row": row,
                    "template_render_fn": template.render,
                    "is_correct": is_correct,
                    "template_render_params": {
                        "question": row["question"],
                        "answers": answers,
                        "is_disjunction": False,
                    },
                }

            items = [
                create_yield(True, [c_a, other_c_a])
                for c_a in cor_as
                for other_c_a in [a for a in cor_as if a != c_a]
            ] + [
                create_yield(False, [i_a, c_a]) for i_a in inc_as for c_a in cor_as
            ] + [
                create_yield(False, [c_a, i_a]) for i_a in inc_as for c_a in cor_as
            ]
            yield from div_items(items, max_per_q)


def disj_ds_generator(max_per_q=None):
    """ Generator for a dataset with two statements in disjunction per question. """

    with torch.no_grad():
        template = env.get_template("question_answers.jinja")
        counter = 0
        for row_id, row in tqdm(list(enumerate(truthfulqa["validation"]))):
            inc_as = row["incorrect_answers"]
            cor_as = row["correct_answers"]

            def create_yield(is_correct, answers):
                nonlocal counter
                counter += 1
                return {
                    "id": counter,
                    "question_id": row_id,
                    "row": row,
                    "template_render_fn": template.render,
                    "is_correct": is_correct,
                    "template_render_params": {
                        "question": row["question"],
                        "answers": answers,
                        "is_disjunction": True,
                    },
                }

            items = (
                [create_yield(True, [c_a, i_a]) for c_a in cor_as for i_a in inc_as]
                + [
                    create_yield(True, [c_a, other_c_a])
                    for c_a in cor_as
                    for other_c_a in [a for a in cor_as if a != c_a]
                ]
                + [
                    create_yield(False, [i_a, other_i_a])
                    for i_a in inc_as
                    for other_i_a in [a for a in inc_as if a != i_a]
                ]
            )
            yield from div_items(items, max_per_q)

In [8]:
report = pd.DataFrame()

In [9]:
def fs_calc_accuracy_for(ds_generator, report, report_index_name, max_per_q=None):
    """ Few-shot accuracy calculation for a given dataset generator."""

    known_questions = set()
    wrong_questions = set()
    count = 0
    correct_n = 0
    pbar = tqdm(ds_generator(max_per_q=max_per_q))
    ds_true_samples_num = 0
    for sample_gen in pbar:
        input_ = sample_gen["template_render_fn"](
            **sample_gen["template_render_params"], label=""
        )
        t_output = tokenizer(input_, return_tensors="pt")
        t_output = {k: t_output[k].to(device) for k in t_output}
        outputs = hf_model(**t_output, output_hidden_states=False)
        pred = outputs.logits[0, -1].softmax(dim=-1)
        token = pred.argmax(-1)
        is_correct = (
            token in true_ids if sample_gen["is_correct"] else token in false_ids
        )
        if sample_gen["is_correct"]:
            ds_true_samples_num += 1
        if is_correct:
            correct_n += 1
            if not sample_gen["question_id"] in wrong_questions:
                known_questions.add(sample_gen["question_id"])
        else:
            wrong_questions.add(sample_gen["question_id"])
            known_questions.discard(sample_gen["question_id"])
        count += 1
        pbar.set_description(
            f"Correct {correct_n}, count {count}, accuracy {correct_n / count:.4}, known {len(known_questions)}"
        )
    assert count > 0
    report.loc[report_index_name, "FS, acc"] = correct_n / count
    report.loc[report_index_name, "FS, # samples"] = count
    report.loc[report_index_name, "FS, # correct"] = correct_n
    report.loc[report_index_name, "FS, # known q."] = len(known_questions)
    report.loc[report_index_name, "FS, DS true/all"] = ds_true_samples_num / count
    return known_questions

In [10]:
# FS - one statement
fs_one_known_qs = fs_calc_accuracy_for(
    one_statement_ds_generator, report, "One statement", max_per_q=EXAMPLES_PER_QUESTION
)
display(report)

100%|██████████| 817/817 [02:01<00:00,  6.74it/s]79: : 1634it [02:01, 13.96it/s]
Correct 974, count 1634, accuracy 0.5961, known 279: : 1634it [02:01, 13.46it/s]


,"FS, acc","FS, # samples","FS, # correct","FS, # known q.","FS, DS true/all"
One statement,0.596083,1634.0,974.0,279.0,0.5


In [11]:
# FS - disjunction
fs_disj_known_qs = fs_calc_accuracy_for(
    disj_ds_generator, report, "Disjunction", max_per_q=EXAMPLES_PER_QUESTION
)
display(report)

100%|██████████| 817/817 [02:11<00:00,  6.23it/s]07: : 1591it [02:11, 11.58it/s]
Correct 591, count 1591, accuracy 0.3715, known 107: : 1591it [02:11, 12.13it/s]


,"FS, acc","FS, # samples","FS, # correct","FS, # known q.","FS, DS true/all"
One statement,0.596083,1634.0,974.0,279.0,0.500000
Disjunction,0.371464,1591.0,591.0,107.0,0.513514


In [12]:
# FS - conjunction
fs_conj_known_qs = fs_calc_accuracy_for(
    conj_ds_generator, report, "Conjunction", max_per_q=EXAMPLES_PER_QUESTION
)
display(report)

100%|██████████| 817/817 [02:08<00:00,  6.37it/s]7: : 1563it [02:08, 12.05it/s]
Correct 694, count 1564, accuracy 0.4437, known 87: : 1564it [02:08, 12.19it/s]


,"FS, acc","FS, # samples","FS, # correct","FS, # known q.","FS, DS true/all"
One statement,0.596083,1634.0,974.0,279.0,0.500000
Disjunction,0.371464,1591.0,591.0,107.0,0.513514
Conjunction,0.443734,1564.0,694.0,87.0,0.477621


In [13]:
report.to_csv(f"data/llama{llama_size}/truthful_qa_report_data.csv")

In [14]:
# Dataset for hidden states:


def create_tensordataset(ds_generator, hf_model, tokenizer, max_per_q=None, layer=-1):
    with torch.no_grad():
        items = list(ds_generator(max_per_q))
        neg_hs = []
        pos_hs = []
        gt_labels = []  # Ground truth labels.
        ids = []
        q_ids = []
        for item in tqdm(items):
            for label in (True, False):
                input_ = item["template_render_fn"](
                    **item["template_render_params"], label=str(label)
                )
                t_output = tokenizer(input_, return_tensors="pt")
                t_output = {k: t_output[k].to(device) for k in t_output}
                output = hf_model(**t_output, output_hidden_states=True)
                hs = output.hidden_states[layer][0, -1].detach()
                if label:
                    pos_hs.append(hs)
                else:
                    neg_hs.append(hs)
            gt_labels.append(item["is_correct"])
            ids.append(item["id"])
            q_ids.append(item["question_id"])

        neg_hs = torch.stack(neg_hs).type(torch.float)
        pos_hs = torch.stack(pos_hs).type(torch.float)
        gt_labels = torch.tensor(gt_labels).type(torch.float)
        ids = torch.tensor(ids).type(torch.int)
        q_ids = torch.tensor(q_ids).type(torch.int)
        return TensorDataset(neg_hs, pos_hs, gt_labels, ids, q_ids)

In [15]:
# Check if datasets exists in files and load them otherwise generate:

if os.path.exists(f"data/llama{llama_size}/truthful_qa_one_hs_ds.pt"):
    one_hs_ds = torch.load(f"data/llama{llama_size}/truthful_qa_one_hs_ds.pt")
else:
    one_hs_ds = create_tensordataset(
        one_statement_ds_generator, hf_model, tokenizer, max_per_q=EXAMPLES_PER_QUESTION
    )
    torch.save(one_hs_ds, f"data/llama{llama_size}/truthful_qa_one_hs_ds.pt")

if os.path.exists(f"data/llama{llama_size}/truthful_qa_disj_hs_ds.pt"):
    disj_hs_ds = torch.load(f"data/llama{llama_size}/truthful_qa_disj_hs_ds.pt")
else:
    disj_hs_ds = create_tensordataset(
        disj_ds_generator, hf_model, tokenizer, max_per_q=EXAMPLES_PER_QUESTION
    )
    torch.save(disj_hs_ds, f"data/llama{llama_size}/truthful_qa_disj_hs_ds.pt")

if os.path.exists(f"data/llama{llama_size}/truthful_qa_conj_hs_ds.pt"):
    conj_hs_ds = torch.load(f"data/llama{llama_size}/truthful_qa_conj_hs_ds.pt")
else:
    conj_hs_ds = create_tensordataset(
        conj_ds_generator, hf_model, tokenizer, max_per_q=EXAMPLES_PER_QUESTION
    )
    torch.save(conj_hs_ds, f"data/llama{llama_size}/truthful_qa_conj_hs_ds.pt")

In [16]:
# Split to train and test:
def split_train_test_ds(ds):
    n = len(ds)
    TRAIN_RATIO = 0.8
    train_n = int(n * TRAIN_RATIO)
    train_ds, test_ds = random_split(ds, [train_n, n - train_n])

    return DataLoader(train_ds, batch_size=32), DataLoader(test_ds, batch_size=32)


one_train_dl, one_test_dl = split_train_test_ds(one_hs_ds)
disj_train_dl, disj_test_dl = split_train_test_ds(disj_hs_ds)
conj_train_dl, conj_test_dl = split_train_test_ds(conj_hs_ds)

In [17]:
class LogisticRegression(pl.LightningModule):
    def __init__(self, input_dim, lr=1e-3, batch_size=32):
        super().__init__()
        self.lr = lr
        self.input_dim = input_dim
        self.fc = nn.Linear(input_dim, 1)
        self.loss = nn.BCEWithLogitsLoss()
        self.batch_size = batch_size

    def training_step(self, batch, batch_idx):
        pos_x, neg_x, y, *_ = batch
        x = neg_x - pos_x
        y_hat = self.fc(x).squeeze()
        loss = self.loss(y_hat, y)
        return loss

    def forward(self, x):
        return self.fc(x)

    def configure_optimizers(self):
        return torch.optim.AdamW(self.parameters(), lr=self.lr)

In [18]:
def calc_LR_accuracy(train_dl, test_dl, report, report_index_name):
    LR_probe = LogisticRegression(train_dl.dataset.dataset.tensors[0].shape[1])
    trainer = pl.Trainer(max_epochs=50)
    trainer.fit(LR_probe, train_dl)
    LR_probe.eval()
    # Accuracy:
    LR_probe = LR_probe.to(device="cpu")
    test_neg_x, test_pos_x, test_y, ids, q_ids = (
        t.cpu() for t in test_dl.dataset.dataset.tensors
    )
    test_x = test_pos_x - test_neg_x
    y_hat = LR_probe(test_x).squeeze().sigmoid()
    y_hat = (y_hat > 0.5).float()
    accuracy = (y_hat == test_y).float().mean()
    # Get indexes of questions with all correct predictions:
    known_questions = set()
    for q_id in q_ids.unique():
        sample_indexes = (q_ids == q_id).nonzero(as_tuple=False).squeeze()
        all_guessed = (y_hat[sample_indexes] == test_y[sample_indexes]).all()
        if all_guessed:
            known_questions.add(q_id.item())

    report.loc[report_index_name, "LR, acc"] = accuracy.item()
    report.loc[report_index_name, "LR, # samples"] = len(test_y)
    report.loc[report_index_name, "LR, # correct"] = sum(y_hat == test_y).item()
    report.loc[report_index_name, "LR, # known q."] = len(known_questions)
    report.loc[report_index_name, "LR, DS true/all"] = test_y.sum().item() / len(test_y)

    return LR_probe, known_questions

In [19]:
# Drop all columns in report that begins with LR:
for col in report.columns:
    if col.startswith("LR"):
        del report[col]

In [20]:
# LR probe - One statement
statement_LR_probe, lr_one_indexes = calc_LR_accuracy(
    one_train_dl, one_test_dl, report, "One statement"
)
report

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/conda/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/logger_connector/logger_connector.py:67: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
You are using a CUDA device ('NVIDIA RTX A6000') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#to

Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=50` reached.


,"FS, acc","FS, # samples","FS, # correct","FS, # known q.","FS, DS true/all","LR, acc","LR, # samples","LR, # correct","LR, # known q.","LR, DS true/all"
One statement,0.596083,1634.0,974.0,279.0,0.500000,0.96022,1634.0,1569.0,754.0,0.5
Disjunction,0.371464,1591.0,591.0,107.0,0.513514,NaN,NaN,NaN,NaN,NaN
Conjunction,0.443734,1564.0,694.0,87.0,0.477621,NaN,NaN,NaN,NaN,NaN


In [21]:
# LR probe - Disjunction statement
disj_LR_probe, lr_disj_indexes = calc_LR_accuracy(
    disj_train_dl, disj_test_dl, report, "Disjunction"
)
report

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type              | Params
-------------------------------------------
0 | fc   | Linear            | 5.1 K 
1 | loss | BCEWithLogitsLoss | 0     
-------------------------------------------
5.1 K     Trainable params
0         Non-trainable params
5.1 K     Total params
0.020     Total estimated model params size (MB)
/opt/conda/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=63` in the `DataLoader` to improve performance.
/opt/conda/lib/python3.10/site-packages/lightning/pytorch/loops/fit_loop.py:293: The number of training batches (40) is smaller than the logging interval Trainer(log_every

Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=50` reached.


,"FS, acc","FS, # samples","FS, # correct","FS, # known q.","FS, DS true/all","LR, acc","LR, # samples","LR, # correct","LR, # known q.","LR, DS true/all"
One statement,0.596083,1634.0,974.0,279.0,0.500000,0.960220,1634.0,1569.0,754.0,0.500000
Disjunction,0.371464,1591.0,591.0,107.0,0.513514,0.920805,1591.0,1465.0,692.0,0.513514
Conjunction,0.443734,1564.0,694.0,87.0,0.477621,NaN,NaN,NaN,NaN,NaN


In [22]:
# LR probe - Conjunction statement
conj_LR_probe, lr_conj_indexes = calc_LR_accuracy(
    conj_train_dl, conj_test_dl, report, "Conjunction"
)
report

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type              | Params
-------------------------------------------
0 | fc   | Linear            | 5.1 K 
1 | loss | BCEWithLogitsLoss | 0     
-------------------------------------------
5.1 K     Trainable params
0         Non-trainable params
5.1 K     Total params
0.020     Total estimated model params size (MB)
/opt/conda/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=63` in the `DataLoader` to improve performance.
/opt/conda/lib/python3.10/site-packages/lightning/pytorch/loops/fit_loop.py:293: The number of training batches (40) is smaller than the logging interval Trainer(log_every

Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=50` reached.


,"FS, acc","FS, # samples","FS, # correct","FS, # known q.","FS, DS true/all","LR, acc","LR, # samples","LR, # correct","LR, # known q.","LR, DS true/all"
One statement,0.596083,1634.0,974.0,279.0,0.500000,0.960220,1634.0,1569.0,754.0,0.500000
Disjunction,0.371464,1591.0,591.0,107.0,0.513514,0.920805,1591.0,1465.0,692.0,0.513514
Conjunction,0.443734,1564.0,694.0,87.0,0.477621,0.928389,1564.0,1452.0,705.0,0.477621


In [23]:
# Save probes

torch.save(
    statement_LR_probe.state_dict(),
    f"data/llama{llama_size}/truthful_qa_one_LR_probe.pt",
)
torch.save(
    disj_LR_probe.state_dict(),
    f"data/llama{llama_size}/truthful_qa_disj_LR_probe.pt",
)
torch.save(
    conj_LR_probe.state_dict(),
    f"data/llama{llama_size}/truthful_qa_conj_LR_probe.pt",
)

In [24]:
class CCS(pl.LightningModule):
    def __init__(
        self, tensor_ds: TensorDataset, lr=1e-3, batch_size=32, var_normalize=False
    ):
        super().__init__()
        self.lr = lr
        self.fc = nn.Sequential(
            nn.Linear(tensor_ds.tensors[0].shape[1], 1), nn.Sigmoid()
        )
        self.batch_size = batch_size
        self.x_mean = [
            tensor_ds.tensors[1].mean(axis=0),  # 0 - neg
            tensor_ds.tensors[0].mean(axis=0),  # 1 - pos
        ]
        self.x_std = [
            tensor_ds.tensors[1].std(axis=0),
            tensor_ds.tensors[0].std(axis=0),
        ]
        # self.running_sum = [
        #    torch.zeros(input_dim, device=device),
        #    torch.zeros(input_dim, device=device),
        # ]
        # self.running_num = [0, 0]
        # self.x_mean = [
        #    torch.zeros(input_dim, device=device),
        #    torch.zeros(input_dim, device=device),
        # ]
        # self.x_std = [
        #    torch.zeros(input_dim, device=device),
        #    torch.zeros(input_dim, device=device),
        # ]
        self.known_questions = set()
        self.var_normalize = var_normalize
        self.Epsilon = 1e-8

    def _normalize(
        self,
        x: torch.Tensor,
        pn_type: int,  # Positive (1) or negative (0).
        update_running=False,
    ):
        """
        Mean-normalizes the data x (of shape (n, d))
        If self.var_normalize, also divides by the standard deviation
        """
        # if update_running:
        #    self.running_sum[pn_type] += x.sum(axis=0)
        #    self.running_num[pn_type] += len(x)
        # self.x_mean[pn_type] = self.running_sum[pn_type] / self.running_num[pn_type]
        # normalized_x = x - self.x_mean[pn_type]
        normalized_x = x - x.mean(axis=0)
        if self.var_normalize:
            normalized_x /= x.std(axis=0) + self.Epsilon
            # self.x_std[pn_type] = (x - self.x_mean[pn_type]).pow(2).sum(
            #    axis=0
            # ) / self.running_num[pn_type]
            #normalized_x /= self.x_std[pn_type] + self.Epsilon

        return normalized_x

    def loss(self, p0: torch.Tensor, p1: torch.Tensor):
        """
        Returns the CCS loss for two probabilities each of shape (n,1) or (n,)
        """
        confidence = (torch.min(p0, p1) ** 2).mean(0)
        consistency = ((p0 - (1 - p1)) ** 2).mean(0)
        return confidence + consistency

    def forward(self, x):
        return self.fc(x)

    def training_step(self, batch, batch_idx):
        neg_x, pos_x, *_ = batch
        pos_x = self._normalize(pos_x, pn_type=1)
        neg_x = self._normalize(neg_x, pn_type=0)
        neg_p = self.fc(neg_x).squeeze()
        pos_p = self.fc(pos_x).squeeze()
        loss = self.loss(neg_p, pos_p)
        self.log("train_loss", loss, on_step=False, on_epoch=True, prog_bar=True)
        return loss

    def test_step(self, batch, batch_idx):
        neg_x, pos_x, y, *_ = batch
        pos_x = self._normalize(pos_x, pn_type=1)
        neg_x = self._normalize(neg_x, pn_type=0)
        with torch.no_grad():
            neg_p = self.fc(neg_x).squeeze()
            pos_p = self.fc(pos_x).squeeze()
        avg_confidence = 0.5 * (pos_p + (1 - neg_p))
        predictions = (avg_confidence.detach() >= 0.5).int()
        acc = (predictions == y).float().mean()
        acc = max(acc, 1 - acc)
        self.log("test_acc", acc, on_step=False, on_epoch=True, prog_bar=True)
        # Known questions:
        for q_id in batch[4].unique():
            q_indexes = (batch[4] == q_id).nonzero(as_tuple=False).squeeze()
            if (predictions[q_indexes] == y[q_indexes]).all():
                q_id = q_id.item()
                self.known_questions.add(q_id)

    def configure_optimizers(self):
        return torch.optim.AdamW(self.parameters(), lr=self.lr)

In [25]:
best_lr = 0.0001
best_bs = 32

In [26]:
# Do sweep:
best_acc = None
train_dl = conj_train_dl
test_dl = conj_test_dl
for lr in (1e-2, 1e-3, 1e-4, 3e-4, 7e-4, 1e-5):
    for bs in (32, 64, 128):
        ccs = CCS(
            train_dl.dataset.dataset,
            lr=lr,
            batch_size=bs,
            var_normalize=False,
        )
        trainer = pl.Trainer(
            max_epochs=5,
            profiler=None,
            logger=False,
            enable_progress_bar=False,
            enable_model_summary=False,
        )
        trainer.fit(ccs, train_dl)
        ccs.eval()

        # Accuracy:
        trainer.test(ccs, test_dl, verbose=False)
        acc = trainer.callback_metrics["test_acc"]
        if best_acc is None or acc > best_acc:
            best_acc = acc
            best_lr = lr
            best_bs = bs
print(f"Best acc: {best_acc:.4}, lr: {best_lr}, bs: {best_bs}")

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/conda/lib/python3.10/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:639: Checkpoint directory /workspace/dlkworks/checkpoints exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/opt/conda/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=63` in the `DataLoader` to improve performance.
`Trainer.fit` stopped: `max_epochs=5` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/opt/conda/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num

Best acc: 0.7476, lr: 0.0003, bs: 32


In [27]:
def calc_CCS_accuracy(train_dl, test_dl, report, report_index_name, tries=6, epochs=50):
    best_probe = None
    best_acc = None
    report.loc[report_index_name, "CCS, random acc"] = 0.0
    r_a_list = []
    t_a_list = []
    for t in range(tries):
        ccs = CCS(
            train_dl.dataset.dataset, lr=best_lr, batch_size=best_bs, var_normalize=False
        )

        # Random accuracy
        trainer = pl.Trainer(max_epochs=epochs)   # Avoid overfit?
        trainer.test(ccs, test_dl, verbose=False)
        trainer.known_questions = set()
        r_a_list += [trainer.callback_metrics[ "test_acc" ].item()]

        # Now train
        trainer.fit(ccs, train_dl)
        ccs.eval()
        # Accuracy:
        trainer.test(ccs, test_dl, verbose=False)
        test_acc = trainer.callback_metrics["test_acc" ].item()
        if best_acc is None or best_acc < test_acc:
            best_probe = ccs
            best_acc = test_acc
        t_a_list += [test_acc]

    report.loc[report_index_name, "CCS, acc, mean"] = np.array(t_a_list).mean()
    report.loc[report_index_name, "CCS, acc, std"] = np.array(t_a_list).std()
    report.loc[report_index_name, "CCS, random acc, mean"] = np.array(r_a_list).mean()
    report.loc[report_index_name, "CCS, random acc, std"] = np.array(r_a_list).std()

    report.loc[report_index_name, "CCS, # samples"] = len(
        test_dl.dataset.dataset.tensors[2]
    )
    report.loc[report_index_name, "CCS, # known q"] = len(best_probe.known_questions)
    report.loc[report_index_name, "CCS, DS true/all"] = test_dl.dataset.dataset.tensors[
        2
    ].sum().item() / len(test_dl.dataset.dataset.tensors[2])

    return best_probe

In [28]:
# CCS probe - One statement
one_ccs_probe = calc_CCS_accuracy(one_train_dl, one_test_dl, report, "One statement")
report

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/opt/conda/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=63` in the `DataLoader` to improve performance.


Testing: |          | 0/? [00:00<?, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type       | Params
------------------------------------
0 | fc   | Sequential | 5.1 K 
------------------------------------
5.1 K     Trainable params
0         Non-trainable params
5.1 K     Total params
0.020     Total estimated model params size (MB)
/opt/conda/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=63` in the `DataLoader` to improve performance.
/opt/conda/lib/python3.10/site-packages/lightning/pytorch/loops/fit_loop.py:293: The number of training batches (41) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=50` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


...

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type       | Params
------------------------------------
0 | fc   | Sequential | 5.1 K 
------------------------------------
5.1 K     Trainable params
0         Non-trainable params
5.1 K     Total params
0.020     Total estimated model params size (MB)


Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=50` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

,"FS, acc","FS, # samples","FS, # correct","FS, # known q.","FS, DS true/all","LR, acc","LR, # samples","LR, # correct","LR, # known q.","LR, DS true/all","CCS, random acc","CCS, acc, mean","CCS, acc, std","CCS, random acc, mean","CCS, random acc, std","CCS, # samples","CCS, # known q","CCS, DS true/all"
One statement,0.596083,1634.0,974.0,279.0,0.500000,0.960220,1634.0,1569.0,754.0,0.500000,0.0,0.721713,0.037412,0.59684,0.044646,1634.0,268.0,0.5
Disjunction,0.371464,1591.0,591.0,107.0,0.513514,0.920805,1591.0,1465.0,692.0,0.513514,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Conjunction,0.443734,1564.0,694.0,87.0,0.477621,0.928389,1564.0,1452.0,705.0,0.477621,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
# CCS probe - Disjunction statement
disj_ccs_probe = calc_CCS_accuracy(disj_train_dl, disj_test_dl, report, "Disjunction")
report

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/opt/conda/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=63` in the `DataLoader` to improve performance.


Testing: |          | 0/? [00:00<?, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type       | Params
------------------------------------
0 | fc   | Sequential | 5.1 K 
------------------------------------
5.1 K     Trainable params
0         Non-trainable params
5.1 K     Total params
0.020     Total estimated model params size (MB)
/opt/conda/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=63` in the `DataLoader` to improve performance.
/opt/conda/lib/python3.10/site-packages/lightning/pytorch/loops/fit_loop.py:293: The number of training batches (40) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=50` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


...

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type       | Params
------------------------------------
0 | fc   | Sequential | 5.1 K 
------------------------------------
5.1 K     Trainable params
0         Non-trainable params
5.1 K     Total params
0.020     Total estimated model params size (MB)


Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=50` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

,"FS, acc","FS, # samples","FS, # correct","FS, # known q.","FS, DS true/all","LR, acc","LR, # samples","LR, # correct","LR, # known q.","LR, DS true/all","CCS, random acc","CCS, acc, mean","CCS, acc, std","CCS, random acc, mean","CCS, random acc, std","CCS, # samples","CCS, # known q","CCS, DS true/all"
One statement,0.596083,1634.0,974.0,279.0,0.500000,0.960220,1634.0,1569.0,754.0,0.500000,0.0,0.721713,0.037412,0.596840,0.044646,1634.0,268.0,0.500000
Disjunction,0.371464,1591.0,591.0,107.0,0.513514,0.920805,1591.0,1465.0,692.0,0.513514,0.0,0.572623,0.012096,0.576803,0.021107,1591.0,231.0,0.513514
Conjunction,0.443734,1564.0,694.0,87.0,0.477621,0.928389,1564.0,1452.0,705.0,0.477621,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
# CCS probe - Conjunction statement
conj_ccs_probe = calc_CCS_accuracy(conj_train_dl, conj_test_dl, report, "Conjunction")
report

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/opt/conda/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=63` in the `DataLoader` to improve performance.


Testing: |          | 0/? [00:00<?, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type       | Params
------------------------------------
0 | fc   | Sequential | 5.1 K 
------------------------------------
5.1 K     Trainable params
0         Non-trainable params
5.1 K     Total params
0.020     Total estimated model params size (MB)
/opt/conda/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=63` in the `DataLoader` to improve performance.
/opt/conda/lib/python3.10/site-packages/lightning/pytorch/loops/fit_loop.py:293: The number of training batches (40) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=50` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


...

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type       | Params
------------------------------------
0 | fc   | Sequential | 5.1 K 
------------------------------------
5.1 K     Trainable params
0         Non-trainable params
5.1 K     Total params
0.020     Total estimated model params size (MB)


Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=50` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

,"FS, acc","FS, # samples","FS, # correct","FS, # known q.","FS, DS true/all","LR, acc","LR, # samples","LR, # correct","LR, # known q.","LR, DS true/all","CCS, random acc","CCS, acc, mean","CCS, acc, std","CCS, random acc, mean","CCS, random acc, std","CCS, # samples","CCS, # known q","CCS, DS true/all"
One statement,0.596083,1634.0,974.0,279.0,0.500000,0.960220,1634.0,1569.0,754.0,0.500000,0.0,0.721713,0.037412,0.596840,0.044646,1634.0,268.0,0.500000
Disjunction,0.371464,1591.0,591.0,107.0,0.513514,0.920805,1591.0,1465.0,692.0,0.513514,0.0,0.572623,0.012096,0.576803,0.021107,1591.0,231.0,0.513514
Conjunction,0.443734,1564.0,694.0,87.0,0.477621,0.928389,1564.0,1452.0,705.0,0.477621,0.0,0.651757,0.053999,0.597977,0.024500,1564.0,242.0,0.477621


In [31]:
# Intersection matrix:

def intersection(a, b):
    im = np.zeros((3, 3))
    for i, a_set in enumerate(a):
        for j, b_set in enumerate(b):
            im[i, j] = len(a_set.intersection(b_set))
    return im

fs_sets = [fs_one_known_qs, fs_disj_known_qs, fs_conj_known_qs]
ccs_sets = [one_ccs_probe.known_questions, disj_ccs_probe.known_questions, conj_ccs_probe.known_questions]
lr_sets = [lr_one_indexes, lr_disj_indexes, lr_conj_indexes]
sets_names = ["FS", "CCS", "LR"]
subsets_names = ["One statement", "Disjunction", "Conjunction"]
for i, sets in enumerate([fs_sets, ccs_sets, lr_sets]):
    for j, sets2 in enumerate([fs_sets, ccs_sets, lr_sets][i+1:]):
        j = j + i + 1
        print(f"Intersection {sets_names[i]} vs {sets_names[j]}:")
        df = pd.DataFrame(
                intersection(sets, sets2),
                columns=subsets_names,
                index=subsets_names,
            )
        display(df)
        np.save(f"data/llama{llama_size}/truthful_qa_{sets_names[i]}_vs_{sets_names[j]}.npy", df.to_numpy())

Intersection FS vs CCS:


,One statement,Disjunction,Conjunction
One statement,93.0,85.0,80.0
Disjunction,32.0,33.0,28.0
Conjunction,31.0,33.0,22.0


Intersection FS vs LR:


,One statement,Disjunction,Conjunction
One statement,260.0,246.0,253.0
Disjunction,95.0,97.0,95.0
Conjunction,80.0,68.0,80.0


Intersection CCS vs LR:


,One statement,Disjunction,Conjunction
One statement,230.0,237.0,237.0
Disjunction,213.0,176.0,199.0
Conjunction,224.0,206.0,200.0


In [32]:
ccs_all_qs = set.union(*ccs_sets)
fs_all_qs = set.union(*fs_sets)
lr_all_qs = set.union(*lr_sets)

# Questions identified by CCS but not by LR or by FS:
ccs_only_qs = ccs_all_qs - fs_all_qs - lr_all_qs
# Questions identified by LR but not by CCS or by FS:
lr_only_qs = lr_all_qs - fs_all_qs - ccs_all_qs
# Questions identified by FS but not by CCS or by LR:
fs_only_qs = fs_all_qs - ccs_all_qs - lr_all_qs

# Display:
print("Unique questions identified by CCS but not by LR or by FS:")
df = pd.DataFrame(
    {
        "CCS only": len(ccs_only_qs),
        "LR only": len(lr_only_qs),
        "FS only": len(fs_only_qs),
        "CCS and LR": len(ccs_all_qs & lr_all_qs),
        "CCS and FS": len(ccs_all_qs & fs_all_qs),
        "LR and FS": len(lr_all_qs & fs_all_qs),
        "CCS and LR and FS": len(ccs_all_qs & lr_all_qs & fs_all_qs),
    },
    index=["# questions"],
).T

display(df)

Unique questions identified by CCS but not by LR or by FS:


,# questions
CCS only,5
LR only,163
FS only,0
CCS and LR,512
CCS and FS,233
LR and FS,370
CCS and LR and FS,233


In [33]:
# Save probes:
torch.save(
    one_ccs_probe.state_dict(),
    f"data/llama{llama_size}/truthful_qa_one_ccs_probe.pt",
)
torch.save(
    disj_ccs_probe.state_dict(),
    f"data/llama{llama_size}/truthful_qa_disj_ccs_probe.pt",
)
torch.save(
    conj_ccs_probe.state_dict(),
    f"data/llama{llama_size}/truthful_qa_conj_ccs_probe.pt",
)

In [34]:
# Save known questions:
with open(f"data/llama{llama_size}/known_questions.npy", "wb") as f:
    np.save(f, fs_one_known_qs)
    np.save(f, fs_disj_known_qs)
    np.save(f, fs_conj_known_qs)
    np.save(f, lr_one_indexes)
    np.save(f, lr_disj_indexes)
    np.save(f, lr_conj_indexes)
    np.save(f, list(one_ccs_probe.known_questions))
    np.save(f, list(disj_ccs_probe.known_questions))
    np.save(f, list(conj_ccs_probe.known_questions))

In [35]:
# Create bar plot with accuracies for Few shot, LR, CCS as bars
# and Random (mean) as a line:

# Multiply all accuracy by 100 and round by 1:
freport = (report * 100).round(1)
fig = go.Figure()
fig.add_trace(
    go.Bar(
        x=freport.index,
        y=freport["FS, acc"],
        name="FS",
        marker_color="rgb(55, 83, 109)",
        text=freport["FS, acc"],
    )
)
fig.add_trace(
    go.Bar(
        x=freport.index,
        y=freport["LR, acc"],
        name="LR",
        marker_color="rgb(26, 118, 255)",
        text=freport["LR, acc"],
    )
)
fig.add_trace(
    go.Bar(
        x=freport.index,
        y=freport["CCS, acc"],
        name="CCS",
        marker_color="rgb(255, 118, 26)",
        text=freport["CCS, acc"],
    )
)
fig.add_trace(
    go.Scatter(
        x=freport.index,
        y=freport["CCS, random acc"],
        name="Random (mean)",
        marker_color="rgb(255, 118, 26)",
    )
)
fig.update_layout(
    title="Accuracy on TruthfulQA dataset in %",
    xaxis_title="Sentence type",
    yaxis_title="Accuracy",
    legend_title="Model",
    template=pt_template,
)

fig.show()

KeyError: 'CCS, acc'

In [ ]:
# Save fig:
fig.write_image(f"data/llama{llama_size}/truthful_qa_report.png")
freport.to_csv(f"data/llama{llama_size}/truthful_qa_report.csv")

: 

In [ ]:
# Save fig:
fig.write_image(f"data/llama{llama_size}/truthful_qa_report.png")
freport.to_csv(f"data/llama{llama_size}/truthful_qa_report.csv")

: 

In [ ]:
# Display random as mean ± std:

print("Table")
freport = report.copy()
freport.rename(
    columns={
        "FS, acc": f"FS, accuracy",
        "LR, acc": f"LR, accuracy",
        "CCS, acc": f"CCS, accuracy",
        "CCS, random acc": f"Random, accuracy",
    },
    inplace=True,
)

# Convert columns with 'accuracy' word into percentage:
for col in freport.columns:
    if "accuracy" in col:
        freport[col] = (freport[col] * 100).round(1)
# Convert columns with '#' char into integer:
for col in freport.columns:
    if "#" in col:
        freport[col] = freport[col].astype(int)


freport.to_csv(f"data/llama{llama_size}/truthful_qa_report_styled.csv")
display(freport)

: 

In [36]:
# Create bar plot with accuracies for Few shot, LR, CCS as bars
# and Random (mean) as a line:

# Multiply all accuracy by 100 and round by 1:
freport = (report * 100).round(1)
fig = go.Figure()
fig.add_trace(
    go.Bar(
        x=freport.index,
        y=freport["FS, acc"],
        name="FS",
        marker_color="rgb(55, 83, 109)",
        text=freport["FS, acc"],
    )
)
fig.add_trace(
    go.Bar(
        x=freport.index,
        y=freport["LR, acc"],
        name="LR",
        marker_color="rgb(26, 118, 255)",
        text=freport["LR, acc"],
    )
)
fig.add_trace(
    go.Bar(
        x=freport.index,
        y=freport["CCS, acc"],
        name="CCS",
        marker_color="rgb(255, 118, 26)",
        text=freport["CCS, acc"],
    )
)
fig.add_trace(
    go.Scatter(
        x=freport.index,
        y=freport["CCS, random acc"],
        name="Random (mean)",
        marker_color="rgb(255, 118, 26)",
    )
)
fig.update_layout(
    title="Accuracy on TruthfulQA dataset in %",
    xaxis_title="Sentence type",
    yaxis_title="Accuracy",
    legend_title="Model",
    template=pt_template,
)

fig.show()

KeyError: 'CCS, acc'

In [37]:
report

,"FS, acc","FS, # samples","FS, # correct","FS, # known q.","FS, DS true/all","LR, acc","LR, # samples","LR, # correct","LR, # known q.","LR, DS true/all","CCS, random acc","CCS, acc, mean","CCS, acc, std","CCS, random acc, mean","CCS, random acc, std","CCS, # samples","CCS, # known q","CCS, DS true/all"
One statement,0.596083,1634.0,974.0,279.0,0.500000,0.960220,1634.0,1569.0,754.0,0.500000,0.0,0.721713,0.037412,0.596840,0.044646,1634.0,268.0,0.500000
Disjunction,0.371464,1591.0,591.0,107.0,0.513514,0.920805,1591.0,1465.0,692.0,0.513514,0.0,0.572623,0.012096,0.576803,0.021107,1591.0,231.0,0.513514
Conjunction,0.443734,1564.0,694.0,87.0,0.477621,0.928389,1564.0,1452.0,705.0,0.477621,0.0,0.651757,0.053999,0.597977,0.024500,1564.0,242.0,0.477621


In [38]:
# Save report

report.to_csv(f"data/llama{llama_size}/truthful_qa_report_data.csv")

In [39]:
!cat data/llama13b/truthful_qa_report_data.csv


,"FS, acc","FS, # samples","FS, # correct","FS, # known q.","FS, DS true/all","LR, acc","LR, # samples","LR, # correct","LR, # known q.","LR, DS true/all","CCS, random acc","CCS, acc, mean","CCS, acc, std","CCS, random acc, mean","CCS, random acc, std","CCS, # samples","CCS, # known q","CCS, DS true/all"
One statement,0.5960832313341493,1634.0,974.0,279.0,0.5,0.9602203369140625,1634.0,1569.0,754.0,0.5,0.0,0.7217125594615936,0.03741233462878217,0.5968399544556936,0.044646107644307466,1634.0,268.0,0.5
Disjunction,0.3714644877435575,1591.0,591.0,107.0,0.5135135135135135,0.920804500579834,1591.0,1465.0,692.0,0.5135135135135135,0.0,0.5726227561632792,0.01209596747411667,0.5768025120099386,0.021106596667242003,1591.0,231.0,0.5135135135135135
Conjunction,0.4437340153452685,1564.0,694.0,87.0,0.47762148337595906,0.928388774394989,1564.0,1452.0,705.0,0.47762148337595906,0.0,0.651757170756658,0.053998965085099195,0.5979765752951304,0.024499935335829913,1564.0,242.0,0.47762148337595906
